In [91]:
import pandas as pd #data analysis
import numpy as np #math
import rdkit #chemical power
from rdkit import Chem 
import argparse #make it a py script
from rdkit.Chem import AllChem, MACCSkeys, Descriptors, DataStructs #fingerprint and lipinsky functionality

In [92]:
#dictionary for library of molecules with associated fingerprints

#smiles
datatable = pd.read_csv("modified2.csv")
datatable.columns = ["SMILES", "h"]
datatable_smiles = datatable.SMILES
datatable_smiles_l = datatable_smiles.tolist()
datatable_smiles_clean = [x for x in datatable_smiles_l if str(x) != 'nan']
#or mols = [Chem.MolFromSmiles(s) for s in datatable_smiles_l]

mols = []
badsmiles = []
for smiles in datatable_smiles_clean:
    if "." not in smiles:
        try:
            mol = Chem.MolFromSmiles(str(smiles))
            mols.append(mol) 
        except:
            print (s)
    #take away

cleaned_mols = [x for x in mols if str(x) != 'nan']

    
#morgan (= ecfp4) fingerprints
#maybe replace r = 2 with different numbers
all_fps = []
for mol in cleaned_mols:
    try:
        all_fps_ind = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048, useChirality=True)
        all_fps.append(all_fps_ind)
    except:
        print ("bad mol found")
array = [np.array(x) for x in all_fps]     

#make a new dataframe with smiles and fingerprint
#remake the smiles since some got lost

new_smiles = [Chem.MolToSmiles(m) for m in cleaned_mols]
data = pd.DataFrame()
data["SMILES"] = new_smiles
data["FP"] = all_fps
data["FP array"] = array

#make it a dictionary

datadict = data.to_dict()   
#data.to_csv("bestdata.csv")

In [93]:
#thats what the function is supposed to do (without argparse)
def fp_test(smi):
  
    mol = (Chem.MolFromSmiles(smi))
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048, useChirality=True)
    molecule_array = np.array(fp)
    return fp
fp_test("CC")


In [94]:
#molecule to molecule fingerprint
#parser for smiles input
parser = argparse.ArgumentParser(
                    prog = 'fingerprint transformation',
                    description = 'calculating ecfp4 fingerprint of molecule in smiles input format',
                    epilog = '...')
parser.add_argument('smi', type=str, help="input SMILES file; use '-' for stdin.")
args = parser.parse_args()

#fingerprint calculation:

def fp():
    args = parser.parse_args()
    smi_data = args.smi
    mol = (Chem.MolFromSmiles(smi_data))
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048, useChirality=True)
    #molecule_array = np.array(fp)
    #return molecule_array
    return fp

usage: fingerprint transformation [-h] smi
fingerprint transformation: error: unrecognized arguments: -f


SystemExit: 2

In [ ]:
#search library on fingerprint similarity and show candidates

#calculating the tanimoto similarity
#add the input molecule!
all_table=pd.DataFrame()
for index, i in enumerate(all_fps):
    for jndex, j in enumerate(all_fps):
        similarity=DataStructs.FingerprintSimilarity(i,j)
        hmap[index,jndex]=similarity
        all_table.loc[mols[index],mols[jndex]]=similarity

In [ ]:
#calculating the relevant properties
table=pd.DataFrame()
for i,mol in enumerate(cleaned_mols):
    Chem.SanitizeMol(mol)
    table.loc[i,'MolWt']=Descriptors.MolWt(mol)
    table.loc[i,'LogP']=Descriptors.MolLogP(mol)
    table.loc[i,'TPSA']=Descriptors.TPSA(mol)

In [ ]:
frames = [table, data]
data_fp_and_properties = pd.concat(frames,axis=1, join='inner')
data_fp_and_properties.to_csv("data_fp_and_properties.csv")
print (max(table.MolWt), min(table.MolWt))
print (max(table.LogP), min(table.LogP))
print (max(table.TPSA), min(table.TPSA))

In [115]:
#dataframe filtering with filters

In [117]:
scores = DataStructs.BulkTanimotoSimilarity(fp_test("CCCCCCC"), all_fps)

#add the scores to the data
data_fp_and_properties["SCORES"] = scores

In [129]:
x = 0.1
candidates = data_fp_and_properties[data_fp_and_properties["SCORE"] > x]
len(candidates)
candidates

,MolWt,LogP,TPSA,SMILES,FP,FP array,SCORE,SCORES
4,404.940000,5.68140,6.48,C[SiH](C)CN(CCCCCCN([Si](C)(C)C)[Si](C)(C)C)[Si](C)(C)C,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.142857,0.142857
27,421.754000,8.71920,20.31,CCCCCCCCCCCCCCCCCCCC(=O)CCN1CCCC(C)C1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.228571,0.228571
28,861.479000,15.60310,70.08,CCCCCCCCC(CCCCCC)CC(=O)CCCCCCN(CCCCCOC(=O)CC(CCCCCC)CCCCCCCC)CCN(CCO)C1CCCCC1,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.142857,0.142857
92,406.387000,1.67122,135.66,CCCCOC(=O)C1OC(Oc2ccc3c(C)cc(=O)oc3c2)=C(O)C(O)[C@@H]1O,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.103448,0.103448
109,877.296000,2.59030,280.51,CCCCCCCC[Si](O)(O)OCN(C(=O)NCO[Si](C)(O)O)C1(CO[Si](O)(O)CCCCCCCC)C(=O)NC(=O)N1CO[Si](O)(O)CCCCCCCC,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.145455,0.145455
111,333.494000,3.61540,64.63,CCC(C)(C)C(=O)OCCNC(=O)OCCCCCCSC,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [ ]:
#main function